# Household energy consumption forecast
#### *- Leveraging Amazon Forecast UI Console -*
---

Previously, we leveraged the Amazon Forecast console to predict household energy consumption. We are now going to leverage the high level APIs from this service to achieve the same: **this notebook takes approximately an hour to run.**

The overall process for using Amazon Forecast is the following:

1. Create a **Dataset Group**, this is the large box that isolates models and the data they are trained on from each other. You can see that as an independant forecasting "project".
1. Create a **Dataset**: in Forecast there are 3 types of dataset, *Target Time Series*, *Related Time Series*, and *Item Metadata*. The *Target Time Series* is required, the others provide additional context that certain algorithms can leverage.
1. **Import data**, this moves the information from S3 into a storage volume where the data can be used for training and validation. You can see this as the ingestion process into the Forecast dataset.
1. **Train a model**, Forecast automates this process for you but you can also select particular algorithms, and you can provide your own hyper parameters or use Hyper Parameter Optimization (HPO) to determine the most performant values for your data.
1. **Deploy a Predictor**, here you are deploying your model so you can use it to generate a forecast.
1. **Query the Forecast**, given a request bounded by time for an item, return the forecast for it. Once you have this you can evaluate its performance or use it to guide your decisions about the future.

In this notebook we will be walking through the first 3 steps outlined above. One additional task that will be done here is to trim part of our training and validation data so that we can measure the accuracy of a forecast against our predictions.

## Table Of Contents
* **Preparation:**
  * Setup
  * Data Preparation
  * Creating the Dataset Group and Dataset
* **Building a predictor:**
  * Create a Predictor
  * Deploy a Predictor
  * Obtain a Forecast
* **Evaluating your forecast:** now is the time to pull down the predictions from this Predictor, and compare them to the actual observed values:
  * Obtaining a Prediction
  * Plotting the Actual Results
  * Plotting the Prediction
  * Comparing the Prediction to Actual Results
* **Cleanup:** after building completing the notebooks you may want to delete the following to prevent any unwanted charges:
  * Forecasts
  * Predictors
  * Datasets
  * Dataset Groups

For more informations about Amazon Forecast APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

In [ ]:
# Cell 01
import boto3
import sagemaker
from sagemaker import get_execution_role

The result will be referred in console operations.

In [ ]:
# Cell 02

BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'forecast-workshop'

region = boto3.Session().region_name
role = get_execution_role()

print(f"Bucket Name: {BUCKET}")
print(f"IAM role: {role}")